In [65]:
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np

# Loading Haar Cascade model for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Transformation pipeline for PyTorch
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the input size of the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading the trained model
model_path = 'face_recognition_model.pth'
CLASSES = 6
label_map = ['Dwayne Johnson', 'Elon Musk', 'Emma Watson', 'Henry Cavill', 'Jennifer Lawrence', 'Zendaya']

model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, CLASSES)
model.load_state_dict(torch.load(model_path))
model.eval()

# Predictions history
predictions_history = []
predictions_colors = []

def display_predictions_history():
    for widget in history_frame.winfo_children():
        widget.destroy()

    for i, (prediction, color) in enumerate(zip(predictions_history, predictions_colors)):
        tk.Label(history_frame, text=prediction, font=("Helvetica", 12), fg=color, bg=graphite_color).pack(anchor='center')

def correct_prediction():
    result_label.config(fg=green_color)
    predictions_colors[-1] = green_color
    display_predictions_history()

def incorrect_prediction():
    result_label.config(fg="red")
    predictions_colors[-1] = "red"
    display_predictions_history()

# Image upload and prediction
def upload_and_predict():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = Image.open(file_path).convert('RGB')
        open_cv_image = np.array(image)
        open_cv_image = open_cv_image[:, :, ::-1].copy()
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

        if len(faces) > 0:
            x, y, w, h = faces[0]
            face = open_cv_image[y:y+h, x:x+w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = Image.fromarray(face)
        else:
            face = image

        face = transform(face).unsqueeze(0)

        with torch.no_grad():
            outputs = model(face)
            _, predicted = torch.max(outputs, 1)
            predicted_class_index = predicted.item()
            predicted_class_name = label_map[predicted_class_index]

        # Add the prediction to the history
        predictions_history.append(predicted_class_name)
        predictions_colors.append("white")
        display_predictions_history()

        # Display the result
        result_label.config(text=f'Predicted person: {predicted_class_name}', fg=white_color, font=("Helvetica", 24, "bold"))

        # Display the uploaded image
        img_display = ImageTk.PhotoImage(image.resize((300, 300)))
        image_label.config(image=img_display)
        image_label.image = img_display

root = tk.Tk()
root.title("Celebrity Face Recognition")

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = int(screen_width)
window_height = int(screen_height)
root.geometry(f"{window_width}x{window_height}")

# THEME
graphite_color = "#4F4F4F"
green_color = "#32CD32"
white_color = "#FFFFFF"
root.configure(bg=graphite_color)

# Description for the user
description_label = tk.Label(root, text="Welcome to Celebrity Face Recognition!\nUpload an image to see the results.\n Press 'Right' if the prediction is correct and 'Wrong' if it's incorrect.",
                             font=("Helvetica", 20, "bold"), fg=white_color, bg=graphite_color, pady=10, padx=10)
description_label.pack(pady=10)

center_frame = tk.Frame(root, bg=graphite_color)
center_frame.pack(fill="both", expand=True, padx=10, pady=10)

# white frame around the image
image_frame = tk.Frame(center_frame, width=310, height=310, bg=white_color)
image_frame.pack(pady=10)
image_frame.pack_propagate(False)

placeholder_frame = tk.Frame(image_frame, width=300, height=300, bg="gray")
placeholder_frame.pack(padx=5, pady=5)
placeholder_frame.pack_propagate(False)

image_label = Label(placeholder_frame)
image_label.pack(fill="both", expand=True)

upload_button = tk.Button(center_frame, text="Upload Image", command=upload_and_predict, font=("Helvetica", 16, "bold"), bg=white_color, fg="black", padx=10, pady=5, bd=3, relief="raised")
upload_button.pack(pady=10)

result_label = tk.Label(center_frame, text="", font=("Helvetica", 24, "bold"), fg="white", bg=graphite_color, pady=10, padx=10)
result_label.pack(pady=10)

# Right/Wrong buttons
buttons_frame = tk.Frame(center_frame, bg=graphite_color)
buttons_frame.pack(pady=5)

correct_button = tk.Button(buttons_frame, text="Right", command=correct_prediction, font=("Helvetica", 14, "bold"), bg=green_color, fg="black", padx=10, pady=5)
correct_button.pack(side="left", padx=10)

wrong_button = tk.Button(buttons_frame, text="Wrong", command=incorrect_prediction, font=("Helvetica", 14, "bold"), bg="red", fg="black", padx=10, pady=5)
wrong_button.pack(side="left", padx=10)

history_label = tk.Label(center_frame, text="Prediction History", font=("Helvetica", 16, "bold"), fg=white_color, bg=graphite_color)
history_label.pack(pady=5)

history_frame = tk.Frame(center_frame, bg=graphite_color)
history_frame.pack(fill="both", expand=True)

root.mainloop()
